# **B2B Sales Prediction**
## The goal is to predict the quarterly sales to each of the 75 customers

In [195]:
import pandas as pd
# Read all CSV files into pandas dataframe
economic_indicators_df = pd.read_csv("EconomicIndicators.csv")
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")


## **Merging helpful Economic Indicator data** :     
### Along with the train data , Economic Indicators can be useful for more accurate prediction.
### Hence, we try to combine them together in dataframe, for both test and train data.

In [196]:
# Here we convert month to Quarter
def month_to_quarter(month):
    if month in range(1, 4):
        return 'Q1'
    elif month in range(4, 7):
        return 'Q2'
    elif month in range(7, 10):
        return 'Q3'
    elif month in range(10, 13):
        return 'Q4'
    elif month in range(13, 16):
        return 'Q5'
    elif month in range(16, 19):
        return 'Q6'
    elif month in range(19, 22):
        return 'Q7'
    elif month in range(22, 25):
        return 'Q8'
    elif month in range(25, 29):
        return 'Q9'
    else:
        return None

In [197]:
# Add column 'Quarter' and remove 'Month' as we need to predict sales based on Quarter
economic_indicators_df['Quarter'] = economic_indicators_df['Month'].apply(month_to_quarter)
economic_indicators_df = economic_indicators_df.drop('Month', axis=1)

In [198]:
# Combine the data for same quarter by averaging the data for each row of same quarter
economic_indicators_df_grouped = economic_indicators_df.groupby('Quarter').mean()
economic_indicators_df_grouped = economic_indicators_df_grouped.reset_index()
print(economic_indicators_df_grouped)

  Quarter  Consumer Sentiment  Interest Rate   PMI  Money Supply  NationalEAI  \
0      Q1           63.133333       1.819736  57.2  20975.900000    48.759005   
1      Q2           57.866667       2.947262  56.3  21475.800000    38.324704   
2      Q3           56.100000       3.229186  51.9  21648.566667    36.429408   
3      Q4           58.800000       3.999262  48.1  21678.400000    41.830051   
4      Q5           64.600000       3.802861  47.8  21539.333333    51.604474   
5      Q6           62.300000       3.692629  48.3  21326.433333    47.580357   
6      Q7           69.600000       4.311674  48.9  20893.733333    58.820032   
7      Q8           64.933333       4.421024  49.2  20846.300000    53.884995   
8      Q9           70.625000       3.894113  50.4  20768.450000    62.824546   

     EastEAI    WestEAI   SouthEAI   NorthEAI  
0  47.290282  48.102173  47.748953  48.759005  
1  36.768411  36.676742  38.124071  39.476987  
2  35.565655  36.163304  35.214299  39.926052

In [199]:
# Join the Economic indicator dataframe with train and test data, so that we can combine them together
train_merged = pd.merge(train_df,economic_indicators_df_grouped, how='inner', left_on='Quarter', right_on='Quarter')
test_merged = pd.merge(test_df, economic_indicators_df_grouped, how='inner', left_on='Quarter', right_on='Quarter')

# Column 'RevenueGrowth' and 'MarketshareChange' have very low value
# We make them larger so that the the values of these columns have better significance
train_merged['RevenueGrowth'] = train_merged['RevenueGrowth'] * 100
train_merged['MarketshareChange'] = train_merged['MarketshareChange'] * 100
test_merged['RevenueGrowth'] = test_merged['RevenueGrowth'] * 100
test_merged['MarketshareChange'] = test_merged['MarketshareChange'] * 100
print(train_merged.head())

   ID Company Quarter  QuickRatio  InventoryRatio  RevenueGrowth  \
0   0   CMP01      Q1        2.02            7.71            5.0   
1   9   CMP02      Q1        2.00            5.46           -7.0   
2  18   CMP03      Q1        0.70             NaN           -8.0   
3  27   CMP04      Q1        2.21            6.88            5.0   
4  36   CMP05      Q1        2.22            3.48          -10.0   

   MarketshareChange Bond rating Stock rating Region  ...   Sales  \
0               -4.0         CCC          Buy  South  ...  1517.0   
1               -2.0         AAA  Strong Sell   West  ...  3376.0   
2                0.0          BB         Hold   East  ...  3270.0   
3               -2.0         CCC         Hold   East  ...  2267.0   
4               -3.0          BB         Sell   East  ...  5697.0   

   Consumer Sentiment  Interest Rate   PMI  Money Supply  NationalEAI  \
0           63.133333       1.819736  57.2       20975.9    48.759005   
1           63.133333       1.

In [200]:
# Check for Null values

print(train_merged.isnull().sum())
print( test_merged.isnull().sum())


ID                      0
Company                 0
Quarter                 0
QuickRatio              0
InventoryRatio        152
RevenueGrowth           0
MarketshareChange       0
Bond rating             0
Stock rating            0
Region                  0
Industry                0
Sales                 150
Consumer Sentiment      0
Interest Rate           0
PMI                     0
Money Supply            0
NationalEAI             0
EastEAI                 0
WestEAI                 0
SouthEAI                0
NorthEAI                0
dtype: int64
ID                     0
Company                0
Quarter                0
QuickRatio             0
InventoryRatio        32
RevenueGrowth          0
MarketshareChange      0
Bond rating            0
Stock rating           0
Region                 0
Industry               0
Consumer Sentiment     0
Interest Rate          0
PMI                    0
Money Supply           0
NationalEAI            0
EastEAI                0
WestEAI         

## **Missing Sales data** :    
### We noticed that some rows of train.csv don't have 'Sales' values, which is our target variable.
### We use regression to fill these missing 'Sales' values
### Below, we used HuberRegression, this was empirical findind after trying out different regressor models
### After noticing that final MAE is least when HuberRegressor is used with grid searching over few hyper params, we use the same for missing 'Sales' prediction in train data

In [201]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

train_data_known_Sales = train_merged.dropna(subset=['Sales'])

# DataFrame with rows where 'Sales' is missing
train_data_missing_Sales = train_merged[train_merged['Sales'].isna()]

xx_train_known_sales = train_data_known_Sales.drop('Sales', axis=1)
yy_train_known_sales = train_data_known_Sales['Sales']

# Define numerical and categorical columns
numerical_cols_sales = xx_train_known_sales.select_dtypes(include=[np.number]).columns

categorical_cols_sales = xx_train_known_sales.select_dtypes(include=['object']).columns

# Pipeline: Standard scaling + Imputation
pipeline_sales = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numerical_cols_sales),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_cols_sales)
])



# Create a full pipeline with the preprocessing steps and the regressor
full_pipeline_sales = Pipeline([
    ('preprocessor', pipeline_sales),
    ('huber', HuberRegressor(max_iter=1000))
])

# Define the parameter grid for HuberRegressor
param_grid_sales = {
    'huber__epsilon': [1.1, 1.35, 1.5],
    'huber__alpha': [0.0001, 0.001, 0.01]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search_sales = GridSearchCV(full_pipeline_sales, param_grid_sales, cv=5, scoring='neg_mean_absolute_error')
grid_search_sales.fit(xx_train_known_sales, yy_train_known_sales)

# Get the best estimator and predict on the test set
best_pipeline_sales = grid_search_sales.best_estimator_

# Predict missing 'Sales' values
train_data_missing_Sales['Sales'] = best_pipeline_sales.predict(train_data_missing_Sales)

# Fill the original DataFrame
train_merged.loc[train_merged['Sales'].isna(), 'Sales'] = train_data_missing_Sales['Sales']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_

In [202]:
# Check for null values after filling missing Sales in train data

print(train_merged.isnull().sum())


ID                      0
Company                 0
Quarter                 0
QuickRatio              0
InventoryRatio        152
RevenueGrowth           0
MarketshareChange       0
Bond rating             0
Stock rating            0
Region                  0
Industry                0
Sales                   0
Consumer Sentiment      0
Interest Rate           0
PMI                     0
Money Supply            0
NationalEAI             0
EastEAI                 0
WestEAI                 0
SouthEAI                0
NorthEAI                0
dtype: int64


In [203]:
# Fill missing values in 'InventoryRatio' for both training and test sets
imputer = SimpleImputer(strategy='mean')
train_merged['InventoryRatio'] = imputer.fit_transform(train_merged[['InventoryRatio']])
test_merged['InventoryRatio'] = imputer.transform(test_merged[['InventoryRatio']])


In [204]:
# Final check for null values and check for description

print(train_merged.isnull().sum())
print(test_merged.isnull().sum())
print(train_merged.describe())
print(test_merged.describe())


ID                    0
Company               0
Quarter               0
QuickRatio            0
InventoryRatio        0
RevenueGrowth         0
MarketshareChange     0
Bond rating           0
Stock rating          0
Region                0
Industry              0
Sales                 0
Consumer Sentiment    0
Interest Rate         0
PMI                   0
Money Supply          0
NationalEAI           0
EastEAI               0
WestEAI               0
SouthEAI              0
NorthEAI              0
dtype: int64
ID                    0
Company               0
Quarter               0
QuickRatio            0
InventoryRatio        0
RevenueGrowth         0
MarketshareChange     0
Bond rating           0
Stock rating          0
Region                0
Industry              0
Consumer Sentiment    0
Interest Rate         0
PMI                   0
Money Supply          0
NationalEAI           0
EastEAI               0
WestEAI               0
SouthEAI              0
NorthEAI              0
dty

In [205]:
from sklearn.model_selection import train_test_split
x = train_merged.drop('Sales', axis=1)
y = train_merged['Sales']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
# Print the data types in X_train
print(X_train.dtypes)

X_train shape: (540, 20)
X_test shape: (135, 20)
y_train shape: (540,)
y_test shape: (135,)
ID                      int64
Company                object
Quarter                object
QuickRatio            float64
InventoryRatio        float64
RevenueGrowth         float64
MarketshareChange     float64
Bond rating            object
Stock rating           object
Region                 object
Industry               object
Consumer Sentiment    float64
Interest Rate         float64
PMI                   float64
Money Supply          float64
NationalEAI           float64
EastEAI               float64
WestEAI               float64
SouthEAI              float64
NorthEAI              float64
dtype: object


## **Regressor Model** :   
### Based on experiments , I found that HuberRegressor, which fits well for data with outliers , is the best fit for given data
### Here we run GridSearch over different hyper params for HuberRegressor and find best model

In [206]:
# Based on experiments , I found that HuberRegressor, which fits well for data with outliers , is the best fit for given data
# Here we run GridSearch over different hyper params for HuberRegressor and find best model
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# Define numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=[np.number]).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Pipeline: Standard scaling + Imputation
pipeline = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numerical_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_cols)
])

# Create a full pipeline with the preprocessing steps and the regressor
full_pipeline = Pipeline([
    ('preprocessor', pipeline),
    ('huber', HuberRegressor(max_iter=1000))
])

# Define the parameter grid for HuberRegressor
param_grid = {
    'huber__epsilon': [1.1, 1.35, 1.5],
    'huber__alpha': [0.0001, 0.001, 0.01]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(full_pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
# Note that we use entire data (X_train + X_test , Y_train + y_test), so that it performs better on Kaggle test data
grid_search.fit(x, y)

# Get the best estimator and predict on the test data
best_pipeline = grid_search.best_estimator_
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Absolute Error:", -grid_search.best_score_)

# Predict on the test set
y_pred = best_pipeline.predict(X_test)

# Evaluate the model using mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_

Best Hyperparameters: {'huber__alpha': 0.0001, 'huber__epsilon': 1.35}
Best Mean Absolute Error: 596.6477425740948
Mean Absolute Error: 555.0115793329742


### Run prediction on test data and create final submission file csv

In [207]:
# Predict on the test_merged DataFrame, which is Kaggle test data
test_predictions = best_pipeline.predict(test_merged)

# Create the result DataFramepd.DataFrame({'ID': test_merged['ID'], 'Sales': (test_prediction)})
result_for_submission = pd.DataFrame({'Id': test_merged['ID'], 'Sales': test_predictions})
print(result_for_submission)
# Creat csv file for submission
result_for_submission.to_csv('result_for_submission.csv', index=False)

      Id        Sales
0      7  1672.879531
1     16  3659.382330
2     25  5403.646922
3     34  2389.800212
4     43  5772.560666
..   ...          ...
145  638  4328.990780
146  647  1674.980026
147  656  5967.521396
148  665  2923.473050
149  674  1695.158551

[150 rows x 2 columns]
